In [2]:
import pandas as pd
import numpy as np

In [102]:
df = pd.read_csv("test_df_2.csv")

In [103]:
# index value is in this temp csv
df.drop(df.columns[0], axis=1, inplace=True)

In [104]:
df

,stop_id,date_time,duration,call_for_service,location,actions_taken,Person Search Consent Given,Property Search Consent Given,basis_for_search,reason_for_stop,result,limited_english,age,gender,Gender Nonconforming,lgbt,race,disabilities,evidence_found
0,55,2/1/2021 10:40,10,No,Foothill Expressway,No Action,No,No,No Search,Traffic Violation,Citation for infraction,No,50,Male,No,No,White,None,None
1,56,2/1/2021 10:54,5,No,San Antonio,No Action,No,No,No Search,Traffic Violation,Warning (verbal or written),Yes,25,Female,No,No,Hispanic/Latino/a,None,None
2,57,2/1/2021 17:27,15,No,foothill exp,No Action,No,No,No Search,Traffic Violation,Warning (verbal or written),No,25,Male,No,No,Hispanic/Latino/a,None,None
3,58,2/1/2021 19:21,10,No,Plaza Central,No Action,No,No,No Search,Traffic Violation,Citation for infraction,No,45,Female,No,No,White,None,None
4,59,2/1/2021 19:27,6,No,SAN ANTONIO RD,No Action,No,No,No Search,Traffic Violation,Warning (verbal or written),No,20,Female,No,No,White,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16172,2327,7/30/2021 22:06,5,No,San Antonio,No Action,No,No,No Search,Traffic Violation,Warning (verbal or written),No,18,Female,No,No,White,None,None
16173,2328,7/31/2021 15:46,8,No,Alvarado,No Action,No,No,No Search,Traffic Violation,Contacted parent/legal guardian or other perso...,No,17,Male,No,No,White,None,None
16174,2328,7/31/2021 15:46,8,No,Alvarado,No Action,No,No,No Search,Traffic Violation,Warning (verbal or written),No,17,Male,No,No,White,None,None
16175,2329,7/31/2021 23:28,3,No,GRANT RD,No Action,No,No,No Search,Traffic Violation,Warning (verbal or written),No,20,Female,No,No,White,None,None


In [53]:
dup_stops = np.unique(df[df["stop_id"].duplicated(keep=False)]["stop_id"].values)

In [150]:
len(dup_stops)

153

In [200]:
def count_individuals(single_stop_df):
    demog_keys= ["age", "race", "gender", "limited_english", "lgbt", "disabilities"]
    stop_id = set(single_stop_df["stop_id"])
    if len(stop_id) == 1:
        stop_id = list(stop_id)[0]
    else:
        print("ERROR: single stop dataframe contains multiple stop_ids! {}".format(stop_id))
        assert(False)
    # get rid of exact duplicate rows.
    orig_records = len(single_stop_df)
    single_stop_df.drop_duplicates(inplace=True)
    uniq_records = len(single_stop_df)
    if uniq_records < orig_records:
        print("Data Issue: stop_id {} has {} identical records.".format(stop_id, orig_records - uniq_records +1))
    if uniq_records == 1:
        return 1
    
    demog=single_stop_df.drop_duplicates(subset=demog_keys, inplace=False)
    demog_counts = demog.nunique()
    individuals = len(demog)
    diffs = demog_counts[demog_counts > 1].keys()
    if individuals > 1:
        print("stop_id {}: Likely {} different individuals, differing by: {}".format(stop_id, individuals, [d for d in diffs]))
    else:
        uniq_res = set(single_stop_df["result"].values)
        if len(uniq_res) > 1 and "No Action" in uniq_res:
            individuals = (single_stop_df["result"].value_counts())["No Action"]+1
            print("stop_id {}: Possibly {}+ individuals, based on different results".format(stop_id, individuals))
        else:
            print("stop_id {}: Likely single individual with multiple actions.".format(stop_id))
    return individuals

In [146]:
individuals = [count_individuals(df[df["stop_id"] == id].copy()) for id in dup_stops]

stop_id 72: Likely 3 different individuals, differing by: ['race']
stop_id 85: Likely single individual with multiple actions.
stop_id 92: Likely single individual with multiple actions.
stop_id 116: Likely single individual with multiple actions.
Data Issue: stop_id 126 has 2 identical records.
Data Issue: stop_id 140 has 21 identical records.
stop_id 140: Likely single individual with multiple actions.
stop_id 141: Likely single individual with multiple actions.
stop_id 1007: Likely single individual with multiple actions.
stop_id 1025: Likely single individual with multiple actions.
Data Issue: stop_id 1030 has 15 identical records.
stop_id 1030: Likely single individual with multiple actions.
stop_id 1048: Likely single individual with multiple actions.
Data Issue: stop_id 1057 has 255 identical records.
stop_id 1057: Likely 2 different individuals, differing by: ['age', 'gender']
Data Issue: stop_id 1089 has 2 identical records.
Data Issue: stop_id 1093 has 17 identical records.
s

Data Issue: stop_id 2315 has 2 identical records.
stop_id 2328: Likely single individual with multiple actions.


In [149]:
len(individuals)

1578

In [140]:
mega = df[df["stop_id"] == 1426]

In [141]:
mega.nunique()

stop_id                          1
date_time                        1
duration                         1
call_for_service                 1
location                         1
actions_taken                    3
Person Search Consent Given      1
Property Search Consent Given    1
basis_for_search                 2
reason_for_stop                  1
result                           2
limited_english                  1
age                              1
gender                           1
Gender Nonconforming             1
lgbt                             1
race                             1
disabilities                     1
evidence_found                   1
dtype: int64

In [142]:
len(mega.drop_duplicates())

12

In [143]:
mega.drop_duplicates()

,stop_id,date_time,duration,call_for_service,location,actions_taken,Person Search Consent Given,Property Search Consent Given,basis_for_search,reason_for_stop,result,limited_english,age,gender,Gender Nonconforming,lgbt,race,disabilities,evidence_found
5441,1426,4/2/2021 1:25,15,No,El CamiNo reap,Person removed from vehicle by order,No,No,Officer Safety/safety of others,Traffic Violation,No Action,No,25,Male,No,No,Hispanic/Latino/a,None,None
5457,1426,4/2/2021 1:25,15,No,El CamiNo reap,Person removed from vehicle by order,No,No,Officer Safety/safety of others,Traffic Violation,Citation for infraction,No,25,Male,No,No,Hispanic/Latino/a,None,None
5505,1426,4/2/2021 1:25,15,No,El CamiNo reap,Person removed from vehicle by order,No,No,No Search,Traffic Violation,No Action,No,25,Male,No,No,Hispanic/Latino/a,None,None
5521,1426,4/2/2021 1:25,15,No,El CamiNo reap,Person removed from vehicle by order,No,No,No Search,Traffic Violation,Citation for infraction,No,25,Male,No,No,Hispanic/Latino/a,None,None
5569,1426,4/2/2021 1:25,15,No,El CamiNo reap,Search of person was conducted,No,No,Officer Safety/safety of others,Traffic Violation,No Action,No,25,Male,No,No,Hispanic/Latino/a,None,None
5585,1426,4/2/2021 1:25,15,No,El CamiNo reap,Search of person was conducted,No,No,Officer Safety/safety of others,Traffic Violation,Citation for infraction,No,25,Male,No,No,Hispanic/Latino/a,None,None
5633,1426,4/2/2021 1:25,15,No,El CamiNo reap,Search of person was conducted,No,No,No Search,Traffic Violation,No Action,No,25,Male,No,No,Hispanic/Latino/a,None,None
5649,1426,4/2/2021 1:25,15,No,El CamiNo reap,Search of person was conducted,No,No,No Search,Traffic Violation,Citation for infraction,No,25,Male,No,No,Hispanic/Latino/a,None,None
5697,1426,4/2/2021 1:25,15,No,El CamiNo reap,No Action,No,No,Officer Safety/safety of others,Traffic Violation,No Action,No,25,Male,No,No,Hispanic/Latino/a,None,None
5713,1426,4/2/2021 1:25,15,No,El CamiNo reap,No Action,No,No,Officer Safety/safety of others,Traffic Violation,Citation for infraction,No,25,Male,No,No,Hispanic/Latino/a,None,None


In [126]:
len(df)

16177

In [128]:
len(df.drop_duplicates())

2117

In [131]:
len(df.drop_duplicates(["stop_id"]))

1425

In [212]:
sample = df.iloc[3]

In [217]:
derp = sample.to_dict()

In [218]:
derp

{'stop_id': 58,
 'date_time': '2/1/2021 19:21',
 'duration': 10,
 'call_for_service': 'No',
 'location': 'Plaza Central',
 'actions_taken': 'No Action',
 'Person Search Consent Given': 'No',
 'Property Search Consent Given': 'No',
 'basis_for_search': 'No Search',
 'reason_for_stop': 'Traffic Violation',
 'result': 'Citation for infraction',
 'limited_english': 'No',
 'age': 45,
 'gender': 'Female',
 'Gender Nonconforming': 'No',
 'lgbt': 'No',
 'race': 'White',
 'disabilities': 'None',
 'evidence_found': 'None'}

In [181]:
demog_keys = ["age", "race", "gender", "limited_english", "lgbt", "disabilities"]

In [182]:
fakedup = sample.copy()

In [188]:
fakedup = pd.concat([sample.copy() for x in range(len(demog_keys)+1)], axis=1, ignore_index=True).T

In [189]:
fakedup.at[1,'age'] = 99
fakedup.at[2,'race'] = 'Alien'
fakedup.at[3,'gender'] = 'non-binary'
fakedup.at[4,'limited_english'] = 'maybe'
fakedup.at[5, 'lgbt'] = 'Yes'
fakedup.at[6, 'disabilities'] = 'some'

In [190]:
fakedup

,stop_id,date_time,duration,call_for_service,location,actions_taken,Person Search Consent Given,Property Search Consent Given,basis_for_search,reason_for_stop,result,limited_english,age,gender,Gender Nonconforming,lgbt,race,disabilities,evidence_found
0,58,2/1/2021 19:21,10,No,Plaza Central,No Action,No,No,No Search,Traffic Violation,Citation for infraction,No,45,Female,No,No,White,None,None
1,58,2/1/2021 19:21,10,No,Plaza Central,No Action,No,No,No Search,Traffic Violation,Citation for infraction,No,99,Female,No,No,White,None,None
2,58,2/1/2021 19:21,10,No,Plaza Central,No Action,No,No,No Search,Traffic Violation,Citation for infraction,No,45,Female,No,No,Alien,None,None
3,58,2/1/2021 19:21,10,No,Plaza Central,No Action,No,No,No Search,Traffic Violation,Citation for infraction,No,45,non-binary,No,No,White,None,None
4,58,2/1/2021 19:21,10,No,Plaza Central,No Action,No,No,No Search,Traffic Violation,Citation for infraction,maybe,45,Female,No,No,White,None,None
5,58,2/1/2021 19:21,10,No,Plaza Central,No Action,No,No,No Search,Traffic Violation,Citation for infraction,No,45,Female,No,Yes,White,None,None
6,58,2/1/2021 19:21,10,No,Plaza Central,No Action,No,No,No Search,Traffic Violation,Citation for infraction,No,45,Female,No,No,White,some,None


In [201]:
count_individuals(fakedup)

stop_id 58: Likely 7 different individuals, differing by: ['limited_english', 'age', 'gender', 'lgbt', 'race', 'disabilities']


7

In [196]:
stop72 = df[df["stop_id"]==1123]

In [202]:
count_individuals(stop72.copy())

Data Issue: stop_id 1123 has 2541 identical records.
stop_id 1123: Likely 2 different individuals, differing by: ['race']


2

In [199]:
stop72.drop_duplicates(subset=demog_keys)

,stop_id,date_time,duration,call_for_service,location,actions_taken,Person Search Consent Given,Property Search Consent Given,basis_for_search,reason_for_stop,result,limited_english,age,gender,Gender Nonconforming,lgbt,race,disabilities,evidence_found
625,1123,2/25/2021 17:51,30,No,El CamiNo Real,Asked for consent to search property,No,Yes,Condition of parole / probation/ PRCS / mandat...,Traffic Violation,Warning (verbal or written),No,30,Male,No,No,White,None,None
629,1123,2/25/2021 17:51,30,No,El CamiNo Real,Asked for consent to search property,No,Yes,Condition of parole / probation/ PRCS / mandat...,Traffic Violation,Warning (verbal or written),No,30,Male,No,No,Black/African American,None,None


In [215]:
bob = pd.Series(derp)

In [216]:
bob

stop_id                                               58
date_time                                 2/1/2021 19:21
duration                                              10
call_for_service                                      No
location                                   Plaza Central
actions_taken                                  No Action
Person Search Consent Given                           No
Property Search Consent Given                         No
basis_for_search                               No Search
reason_for_stop                        Traffic Violation
result                           Citation for infraction
limited_english                                       No
age                                                   45
gender                                            Female
Gender Nonconforming                                  No
lgbt                                                  No
race                                               White
disabilities                   